## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is, essentially, the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

* Describe RDDs as fundamental storage units in the Spark computing environment
* Create RDDs from Python collections
* Set number of partitions for parallelizing RDDs
* Review an RDD's dependency graph at different stages of processing. 
* Apply the map(func) transformation to a given function on all elements of an RDD in different partitions
* Use collect() action to trigger the processing stage of spark's lazy evaluation
* Use count() action to calculate the number of elements of a parallelized RDD
* Use filter(func) to filter unwanted data from RDDs
* Develop an understanding of Python's lambda functions for RDDs processing


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies will performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified.
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the Database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new data set from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base data set (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the Driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some of key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [1]:
pwd

'/home/jovyan'

In [2]:
data = list(range(1,1001))
len(data)

#1000

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use 'local[*]' as the master.

In [3]:
import pyspark
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `parallelize` method to create an rdd will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [4]:
rdd = sc.parallelize(data, numSlices=10)
print(type(rdd))
# <class 'pyspark.rdd.RDD'>

<class 'pyspark.rdd.RDD'>


Determine how many partitions are being used with this RDD with the getNumPartitions method.

In [5]:
rdd.getNumPartitions()
# 10

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first n items in the RDD
* `top`: returns the top n items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [7]:
# count
rdd.count()

1000

In [8]:
# first
rdd.first()

1

In [9]:
# take
rdd.take(5)

[1, 2, 3, 4, 5]

In [11]:
# top
rdd.top(5)

[1000, 999, 998, 997, 996]

In [12]:
# collect
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 be a random number from 0-1.

In [13]:
import random
import numpy as np

nums = np.array(range(1,1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([7.46938799e-01, 1.93583783e+00, 4.35581582e-02, 1.78987279e+00,
       3.67861191e+00, 4.42991456e+00, 5.16083681e+00, 7.82088513e+00,
       8.97996511e+00, 1.59827695e+00, 5.49239239e+00, 1.06862123e+00,
       5.97531516e+00, 8.13042684e+00, 1.39155282e+01, 1.46739038e+01,
       7.81199570e+00, 1.75976201e+01, 1.36372390e+01, 1.90009988e+01,
       1.83653523e+01, 2.07846861e+01, 1.04050970e+01, 2.23517980e+01,
       9.89097081e-01, 1.19507656e+01, 1.97349086e+01, 8.90471146e+00,
       1.90512183e+01, 1.53134643e+01, 2.95846025e+01, 2.17090975e+01,
       2.84730660e+01, 2.45485658e+01, 2.26547480e+01, 2.65872555e+00,
       1.53893725e+01, 2.38932375e+00, 1.47476356e+01, 3.13392982e+01,
       3.61344008e+01, 2.07396298e+00, 1.62767617e+01, 1.89920932e+00,
       3.02684606e+01, 4.36396973e+01, 1.84886164e+01, 3.17894811e-01,
       3.76097962e+01, 1.70098213e+01, 4.62272262e+00, 4.96370450e+01,
       2.78251090e+01, 5.15373131e+01, 4.45674519e+01, 1.52466610e+01,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [14]:
price_items = sc.parallelize(sales_figures, numSlices=10)
price_items.take(5)


[0.7469387992490036,
 1.9358378343731981,
 0.04355815820836739,
 1.7898727876834188,
 3.678611905282454]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax` that returns the amount of money our company will receive after the sale tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the map method and assign it to a variable `renenue_minus_tax`

In [16]:
def sales_tax(num):
    return num * 0.92

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [17]:
# perform action to retrieve rdd values
revenue_minus_tax.take(10)

[0.6871836953090834,
 1.7809708076233424,
 0.040073505551698,
 1.6466829646687453,
 3.3843229528598577,
 4.075521392028616,
 4.747969860709471,
 7.195214322666465,
 8.26156789901766,
 1.4704147958460831]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a map method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [18]:
discounted = price_items.map(lambda x : x * 0.9)

In [19]:
discounted.take(10)

[0.6722449193241032,
 1.7422540509358784,
 0.039202342387530655,
 1.610885508915077,
 3.310750714754209,
 3.9869231008975587,
 4.644753124607091,
 7.038796619999803,
 8.081968596865101,
 1.4384492568059508]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff use a collection method to see the 15 costliest items.

In [20]:
price_items.map(sales_tax).map(lambda x : x * 0.9).top(15)

[784.2024012760754,
 779.5522898886078,
 778.9353445845481,
 766.571936697746,
 760.4613238732272,
 757.3598035003217,
 754.5180451846823,
 752.3860635599525,
 747.0660213846147,
 747.0124258256506,
 746.11441115387,
 736.9027018525672,
 734.4536106334355,
 723.4739044773044,
 718.7965426588595]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [21]:
discounted.toDebugString()

b'(10) PythonRDD[10] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[5] at parallelize at PythonRDD.scala:195 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use flatMap rather than a simple map. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will a tuple with (original price, post-discount price).

In [22]:
mapped =price_items.map(lambda x: (x, x*0.92 *0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.7469387992490036, 0.618465325778175), (1.9358378343731981, 1.6028737268610083), (0.04355815820836739, 0.0360661549965282), (1.7898727876834188, 1.4820146682018709), (3.678611905282454, 3.0458906575738722), (4.429914556552843, 3.6679692528257544), (5.16083680511899, 4.273172874638524), (7.820885133333114, 6.475692890399818), (8.97996510762789, 7.435411109115894), (1.598276952006612, 1.323373316261475)]


Note that we have 1000 tuples created to our specification. Let's take a look at how flatMap differs in its implementation. Use the `flatMap` method with the same function you created above.

In [23]:
flat_mapped = price_items.flatMap(lambda x: (x, x*0.92 *0.9))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.7469387992490036, 0.618465325778175, 1.9358378343731981, 1.6028737268610083, 0.04355815820836739, 0.0360661549965282, 1.7898727876834188, 1.4820146682018709, 3.678611905282454, 3.0458906575738722]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use flatmap rather than map in order to properly reduce to our specifications. This is not one of those instances, but int he upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the filter method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a filter function to meet the consultant's suggestion's specifications. set RDD = `selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [24]:
selected_items = discounted.filter(lambda x: x>300)
selected_items.count()
# calculate total remaining in inventory 


311

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of all of its items after they've reduced their inventory. Use a reduce method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [25]:
selected_items.reduce(lambda x,y :x + y)

152890.34227243473

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [26]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1,50),x))

sales_data.take(7)

[(35, 308.016544484907),
 (19, 303.4971055569342),
 (9, 300.541567710968),
 (50, 315.4006341578518),
 (42, 324.08325895890255),
 (24, 333.1261359677368),
 (30, 318.4448477799752)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called __reduceByKey__ to perform reducing operations while grouping by keys. After you have calculated the total, use the __sortBy__ method on the RDD to rank the users from the highest spending to the least spending.



In [27]:
# calculate how much each user spent
total_spent = sales_data.reduceByKey(lambda x,y :x + y)
total_spent.take(10)

[(20, 673.2738610228431),
 (30, 4766.567488363211),
 (50, 3339.483316892571),
 (40, 3935.8803178591315),
 (10, 644.0607039493975),
 (1, 3874.088389593729),
 (21, 2023.643083420257),
 (31, 1013.4453009108147),
 (41, 2764.5918426116295),
 (11, 1563.510182787038)]

In [29]:
total_spent.sortBy(lambda x: x[1],ascending = False).collect()# sort the users from highest to lowest spenders

[(25, 6029.271624116289),
 (16, 5260.554076532657),
 (15, 4894.277903807271),
 (38, 4824.8282175843),
 (30, 4766.567488363211),
 (32, 4421.92203031063),
 (24, 4409.274639311356),
 (42, 4387.997521695305),
 (36, 4113.217779896327),
 (2, 3984.9655924709123),
 (40, 3935.8803178591315),
 (39, 3901.0524269288935),
 (3, 3876.155760433193),
 (1, 3874.088389593729),
 (28, 3809.123161353531),
 (45, 3792.588665565625),
 (44, 3698.0772665987183),
 (19, 3605.186687613601),
 (4, 3545.748896888277),
 (23, 3523.398599327182),
 (29, 3477.4998411573706),
 (9, 3449.135818929407),
 (37, 3368.7180133636816),
 (13, 3340.5939146111878),
 (50, 3339.483316892571),
 (8, 3256.331852037968),
 (34, 3246.9668652087403),
 (27, 3232.935581702597),
 (46, 3036.6810338946616),
 (49, 2777.6273258186084),
 (41, 2764.5918426116295),
 (18, 2761.4137992168676),
 (47, 2643.209894531671),
 (48, 2489.560757342903),
 (17, 2454.2109972513745),
 (12, 2400.6998340100154),
 (33, 2225.890957375334),
 (7, 2158.904514516065),
 (21, 20

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [30]:
total_items = sales_data.countByKey()
sorted(total_items.items(),key=lambda kv:kv[1],reverse=True)

[(21, 12),
 (37, 12),
 (47, 10),
 (48, 10),
 (19, 10),
 (7, 9),
 (15, 9),
 (9, 9),
 (38, 9),
 (27, 9),
 (36, 9),
 (30, 8),
 (35, 8),
 (29, 8),
 (4, 8),
 (40, 7),
 (45, 7),
 (3, 7),
 (22, 7),
 (39, 6),
 (31, 6),
 (50, 6),
 (2, 6),
 (12, 6),
 (10, 6),
 (11, 6),
 (16, 6),
 (13, 5),
 (44, 5),
 (20, 5),
 (34, 5),
 (8, 5),
 (28, 5),
 (6, 5),
 (26, 5),
 (32, 5),
 (42, 5),
 (17, 5),
 (43, 4),
 (41, 4),
 (1, 4),
 (23, 4),
 (18, 4),
 (14, 4),
 (33, 4),
 (25, 3),
 (5, 3),
 (24, 2),
 (46, 2),
 (49, 2)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD, and extracting lineage and of an RDD in a spark application. We also used transformations and actions to perform calculations across RDDs on a distributed setup. Up next, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
